In [1]:
import os
from pyspark.sql import SparkSession

### Spark Configurations
To see all configurable parameters, open [Spark Configuration](https://spark.apache.org/docs/latest/configuration.html)

In [2]:
spark_session = SparkSession.builder\
    .master("spark://192.168.2.216:7077")\
    .appName("Group 8: Bitcoin Analysis From Reddit Comments")\
    .config("spark.executor.memory", "2g")\
    .getOrCreate()

spark_context = spark_session.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/17 23:44:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark_session.read.json("hdfs://192.168.2.216:9000/user/root/reddit_data/RC_2007-03")
df.take(2)

[Row(author='[deleted]', author_flair_css_class=None, author_flair_text=None, body='[deleted]', controversiality=0, created_utc=1172707213, distinguished=None, edited='false', gilded=0, id='c174td', link_id='t3_1742j', parent_id='t3_1742j', retrieved_on=1473819365, score=15, stickied=False, subreddit='programming', subreddit_id='t5_2fwo', ups=15),
 Row(author='jvance', author_flair_css_class=None, author_flair_text=None, body='Flouride, good.\r\n\r\nHolistic Dentistry wackos and the Weston A. Price Foundation, bad.', controversiality=0, created_utc=1172707269, distinguished=None, edited='false', gilded=0, id='c174tg', link_id='t3_170ie', parent_id='t1_c174mz', retrieved_on=1473819365, score=3, stickied=False, subreddit='reddit.com', subreddit_id='t5_6', ups=3)]

In [4]:
def count_special_words(path, special_words):
    sc = spark_session.sparkContext
    
    
    rdd = sc.textFile(path)
    words_rdd = rdd.flatMap(lambda line: line.split()).map(lambda word: word.lower())
    special_words_rdd = words_rdd.filter(lambda word: word in special_words).map(lambda word: (word, 1))
    counts_rdd = special_words_rdd.reduceByKey(lambda x, y: x + y)
    counts_dict = dict(counts_rdd.collect())
    
    sc.stop()
    
    return counts_dict

In [6]:
count_special_words("hdfs://192.168.2.208:9000/reddit_data/RC_2009-04", ['is', 'are'])

Py4JJavaError: An error occurred while calling o32.partitions.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: hdfs://192.168.2.208:9000/reddit_data/RC_2009-04
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:304)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:244)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:332)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:208)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:292)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:288)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:292)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:288)
	at org.apache.spark.api.java.JavaRDDLike.partitions(JavaRDDLike.sc

In [ ]:
for line in rdd.take(2):
    print(line)

In [10]:
spark_session.stop()